# Train ML

> A collection of machine learning tools

In [ ]:
#| default_exp train

In [ ]:
#| hide
import sys
sys.path.append("/notebooks/katlas")
from nbdev.showdoc import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from sklearn import set_config
from katlas.core import Data
from katlas.feature import *
from fastbook import *
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import spearmanr,pearsonr
from sklearn.model_selection import *
from pathlib import Path
from sklearn.metrics import mean_squared_error
import math
from scipy.stats import spearmanr, pearsonr
from joblib import dump, load
from sklearn.linear_model import *
from sklearn.svm import *
from sklearn.ensemble import *

/usr/local/lib/python3.9/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.9/dist-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.9/dist-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument 

In [ ]:
#| export
from sklearn import set_config
set_config(transform_output="pandas")

## Splitter

In [ ]:
#| export
def get_splits(df, # df contains info for split
               stratified=None, # colname to make stratified kfold; sampling from different groups
               group=None, # colname to make group kfold; test and train are from different groups
               nfold=5,
              seed=123):
    # train_idx, test_idx = None, None
    
    splits = []
    if stratified is not None and group is None:
        kf = StratifiedKFold(nfold, shuffle=True, random_state=seed)
        for split in kf.split(df.index, df[stratified]):
            splits.append(split)
        print(kf)
        print(f'# kinase {stratified} in train set: {df.loc[split[0]][stratified].unique().shape[0]}')
        print(f'# kinase {stratified} in test set: {df.loc[split[1]][stratified].unique().shape[0]}')
        
    elif group is not None and stratified is None:
        kf = GroupKFold(nfold)
        for split in kf.split(df.index, groups=df[group]):
            splits.append(split)
            
        print(kf)
        print(f'# kinase {group} in train set: {df.loc[split[0]][group].unique().shape[0]}')
        print(f'# kinase {group} in test set: {df.loc[split[1]][group].unique().shape[0]}')
        
    elif stratified is not None and group is not None:
        kf = StratifiedGroupKFold(nfold, shuffle=True, random_state=seed)
        for split in kf.split(df.index, groups=df[group], y=df[stratified]):
            splits.append(split)
            
        print(kf)    
        print(f'# kinase {stratified} in train set: {df.loc[split[0]][stratified].unique().shape[0]}')
        print(f'# kinase {stratified} in test set: {df.loc[split[1]][stratified].unique().shape[0]}')
    else:
        raise ValueError("Either 'stratified' or 'group' argument must be provided.")
        
        
    print('---------------------------')
    print(f'# kinase in train set: {df.loc[split[0]].kinase.unique().shape[0]}')
    
    print('---------------------------')
    print(f'# kinase in test set: {df.loc[split[1]].kinase.unique().shape[0]}')
    print('---------------------------')
    print(f'test set: {df.loc[split[1]].kinase.unique()}')
    
    return splits

In [ ]:
df = pd.read_parquet('train/scaled_t5.parquet')

info = Data.get_kinase_info_full()
info = info.query('in_paper == 1')
info = df[['kinase']].merge(info,'left')

In [ ]:
df.head(2)

,kinase,-1A,-1C,-1D,-1E,-1F,-1G,-1H,-1I,-1K,-1L,-1M,-1N,-1P,-1Q,-1R,-1S,-1T,-1V,-1W,-1Y,-1t,-1y,-2A,-2C,-2D,-2E,-2F,-2G,-2H,-2I,-2K,-2L,-2M,-2N,-2P,-2Q,-2R,-2S,-2T,-2V,-2W,-2Y,-2t,-2y,-3A,-3C,-3D,-3E,-3F,-3G,-3H,-3I,-3K,-3L,-3M,-3N,-3P,-3Q,-3R,-3S,-3T,-3V,-3W,-3Y,-3t,-3y,-4A,-4C,-4D,-4E,-4F,-4G,-4H,-4I,-4K,-4L,-4M,-4N,-4P,-4Q,-4R,-4S,-4T,-4V,-4W,-4Y,-4t,-4y,-5A,-5C,-5D,-5E,-5F,-5G,-5H,-5I,-5K,-5L,-5M,-5N,-5P,-5Q,-5R,-5S,-5T,-5V,-5W,-5Y,-5t,-5y,1A,1C,1D,1E,1F,1G,1H,1I,1K,1L,1M,1N,1P,1Q,1R,1S,1T,1V,1W,1Y,1t,1y,2A,2C,2D,2E,2F,2G,2H,2I,2K,2L,2M,2N,2P,2Q,2R,2S,2T,2V,2W,2Y,2t,2y,3A,3C,3D,3E,3F,3G,3H,3I,3K,3L,3M,3N,3P,3Q,3R,3S,3T,3V,3W,3Y,3t,3y,4A,4C,4D,4E,4F,4G,4H,4I,4K,4L,4M,4N,4P,4Q,4R,4S,4T,4V,4W,4Y,4t,4y,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,...,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,AAK1,0.946369,0.750092,0.396778,0.359132,1.338569,1.735594,1.691909,1.068228,3.021196,1.607434,1.361080,1.065893,2.791637,0.976300,2.269404,0.646482,0.773602,0.962921,1.081

In [ ]:
info.head(3)

,kinase,ID_coral,uniprot,ID_HGNC,group,family,subfamily_coral,subfamily,in_paper,pseudo,species_paper,uniprot_paper,range,human_uniprot_sequence,full_sequence_paper,sequence,category,category_big,cluster,length,kinasecom_domain,hdbscan
0,AAK1,AAK1,Q2M2I8,AAK1,Other,NAK,None,NAK,1,0,human,Q2M2I8,aa27-365,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAP...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAP...,TSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIA,NAK,NAK,18.0,339,VTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYF,3.0
1,ACVR2A,ACTR2,P27037,ACVR2A,TKL,STKR,STKR2,STKR2,1,0,human,P27037,aa162-end,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTGVEPCYGDKDKRRHCFATWKNISGSIEIVKQGCWLDDINCYDRTDCVEKKDSPEVYFCCCEGNMCNEKFSYFPEMEVTQPTSNPVTPKPPYYNILLYSLVPLMLIAGIVICAFWVYRHHKMAYPPVLVPTQDPGPPPPSPLLGLKPLQLLEVKARGRFGCVWKAQLLNEYVAVKIFPIQDKQSWQNEYEVYSLPGMKHENILQFIGAEKRGTSVDVDLWLITAFHEKGSLSDFLKANVVSWNELCHIAETMARGLAYLHEDIPGLKDGHKPAISHRDIKSKNVLLKNNLTACIADFGLALKFEAGKSAGDTHGQVGTRRYMAPEVLEGAINFQRDAFLRIDMYAMGLVLWELASRCTAADGPVDEYMLPFEEEIGQHPSLEDMQEVVVHKKKRPVLRDYWQKHAGMAMLCETIEECWDHDAEARLSAGCVGERITQMQRLTNIITTEDIVTVVTMVTNVDFPPKESSL,MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTGVEPCYGDKDKRRHCFATWKNISGSIEIVKQGCWLDDINCYDRTDCVEKKDSPEVYFCCCEGNMCNEKFSYFPEMEVTQPTSNPVTPKPPYYNILLYSLVPLMLIAGIVICAFWVYRHHKMAYPPVLVPTQDPGPPPPSPLLGLKPLQLLEVKARGRFGCVWKAQLLNEYVAVKIFPIQDKQSWQNEYEVYSLPGMKHENILQFIGAEKRGTSVDVDLWLITAFHEKGSLSDFLKANVVSWNELCHIAETMARGLAYLHEDIPGLKDGHKPAISHRDIKSKNVLLKNNLTACIADFGLALKFEAGKSAGDTHGQVGTRRYMAPEVLEGAINFQRDAFLRIDMYAMGLVLWELASRCTAADGPVDEYMLPFEEEIGQHPSLEDMQEVVVHKKKRPVLRDYWQKHAGMAMLCETIEECWDHDAEARLSAGCVGERITQMQRLTNIITTEDIVTVVTMVTNVDFPPKESSL,RHHKMAYPPVLVPTQDPGPPPPSPLLGLKPLQLLEVKARGRFGCVWKAQLLNEYVAVKIFPIQDKQSWQNEYEVYSLPGMKHENILQFIGAEKRGTSVDVDLWLITAFHEKGSLSDFLKANVVSWNELCHIAETMARGLAYLHEDIPGLKDGHKPAISHRDIKSKNVLLKNNLTACIADFGLALKFEAGKSAGDTHGQVGTRRYMAPEVLEGAINFQRDAFLRIDMYAMGLVLWELASRCTAADGPVDEYMLPFEEEIGQHPSLEDMQEVVVHKKKRPVLRDYWQKHAGMAMLCETIEECWDHDAEARLSAGCVGERITQMQRLTNIITTEDIVTVVTMVTNVDFPPKESSL,TGFBR,acidophilic,23.0,352,LQLLEVKARGRFGCVWKAQLLNEYVAVKIFPIQDKQSWQNEYEVYSLPGMKHENILQFIGAEKRGTSVDVDLWLITAFHEKGSLSDFLKANVVSWNELCHIAETMARGLAYLHEDIPGLKDGHKPAISHRDIKSKNVLLKNNLTACIADFGLALKFEAGKSAGDTHGQVGTRRYMAPEVLEGAINFQRDAFLRIDMYAMGLVLWELASRCTAADGPVDEYMLPFEEEIGQHPSLEDMQEVVVHKKKRPVLRDYWQKHAGMAMLCETIEECWDHDAEARLSAGCVGERI,3.0
2,ACVR2B,ACTR2B,Q13705,ACVR2B,TKL,STKR,STKR2,STKR2,1,0,human,Q13705,aa161-end,MTAPWVALALLW

Make sure the info and training df share same index

In [ ]:
splits = get_splits(info, stratified = 'hdbscan') # use info to split the data

StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
# kinase hdbscan in train set: 8
# kinase hdbscan in test set: 8
---------------------------
# kinase in train set: 243
---------------------------
# kinase in test set: 60
---------------------------
test set: ['ACVR2A' 'ACVR2B' 'AKT1' 'AMPKA2' 'ANKRD3' 'AURA' 'BMPR1B' 'BMPR2' 'BRSK2' 'CAMK1D' 'CAMK2D' 'CDK17' 'CDK7' 'CHAK2' 'CK1A2' 'CLK3' 'DCAMKL1' 'DYRK3' 'ERK2' 'ERK7' 'GRK3' 'GRK6' 'HASPIN' 'HIPK3'
 'IRAK1' 'IRAK4' 'IRE1' 'JNK1' 'LOK' 'LRRK2' 'MEK1' 'MEKK1' 'MEKK2' 'MNK2' 'MOK' 'MRCKA' 'MSK1' 'MST4' 'MTOR' 'NIK' 'NUAK1' 'NUAK2' 'P38B' 'PAK3' 'PKACG' 'PKCE' 'PKCG' 'PRKD1' 'PRKD2' 'PRPK' 'RIPK1'
 'RSK2' 'SIK' 'SRPK1' 'TAO2' 'TGFBR1' 'TLK2' 'ULK1' 'WNK1' 'YSK4']


In [ ]:
len(splits)

5

In [ ]:
#| export
def split_data(df, # dataframe of values
               feat_col, # feature columns
               target_col, # target columns
               split # one of the split in splits
              ):
    
    X_train = df.loc[split[0]][feat_col]
    y_train = df.loc[split[0]][target_col]
    
    X_test = df.loc[split[1]][feat_col]
    y_test = df.loc[split[1]][target_col]
    
    return X_train, y_train, X_test, y_test

In [ ]:
feat_col = df.columns[199:]
target_col = df.columns[1:199]

In [ ]:
# feat_col = ['position'] + df.columns.tolist()[5:]
# target_col = ['target']

In [ ]:
# split data
X_train, y_train, X_test, y_test = split_data(df, feat_col, target_col, splits[0])

In [ ]:
print(f'X_train shape is : {X_train.shape}')
print(f'y_train shape is : {y_train.shape}')
print(f'X_test shape is : {X_test.shape}')
print(f'y_test shape is : {y_test.shape}')

X_train shape is : (242, 1024)
y_train shape is : (242, 198)
X_test shape is : (61, 1024)
y_test shape is : (61, 198)


## ML Trainer

In [ ]:
#| export
def train_ml(df, # dataframe of values
             feat_col, # feature columns
             target_col, # target columns
             split, # one split in splits
             model,  # a sklearn models
             save = None, # file (.joblib) to save, e.g. 'model.joblib'
             params={},
            ):
    
    " Train one split of data. Need to specify dataframe, feature columns, target columns, split, and which sklearn models to use"
    
    # split data
    X_train, y_train, X_test, y_test = split_data(df, feat_col, target_col, split)
    
    # Fit the model
    model.fit(X_train, y_train, **params) # better convert y_train to numpy array and flatten
    print(model)
    
    if save is not None:
        # Save the model to a file
        # joblib.dump(model, save)
        dump(model, save)
    
    # Predict
    y_pred = model.predict(X_test) # X_test is dataframe, y_pred is numpy array
    y_pred = pd.DataFrame(y_pred,index=y_test.index, columns = y_test.columns)
    
    
    return y_test, y_pred #two dataframes

In [ ]:
y_test, y_pred = train_ml(df, feat_col, target_col, splits[0],LinearRegression())

LinearRegression()


In [ ]:
y_test.head(2) # ground truth

,-1A,-1C,-1D,-1E,-1F,-1G,-1H,-1I,-1K,-1L,-1M,-1N,-1P,-1Q,-1R,-1S,-1T,-1V,-1W,-1Y,-1t,-1y,-2A,-2C,-2D,-2E,-2F,-2G,-2H,-2I,-2K,-2L,-2M,-2N,-2P,-2Q,-2R,-2S,-2T,-2V,-2W,-2Y,-2t,-2y,-3A,-3C,-3D,-3E,-3F,-3G,-3H,-3I,-3K,-3L,-3M,-3N,-3P,-3Q,-3R,-3S,-3T,-3V,-3W,-3Y,-3t,-3y,-4A,-4C,-4D,-4E,-4F,-4G,-4H,-4I,-4K,-4L,-4M,-4N,-4P,-4Q,-4R,-4S,-4T,-4V,-4W,-4Y,-4t,-4y,-5A,-5C,-5D,-5E,-5F,-5G,-5H,-5I,-5K,-5L,-5M,-5N,-5P,-5Q,-5R,-5S,-5T,-5V,-5W,-5Y,-5t,-5y,1A,1C,1D,1E,1F,1G,1H,1I,1K,1L,1M,1N,1P,1Q,1R,1S,1T,1V,1W,1Y,1t,1y,2A,2C,2D,2E,2F,2G,2H,2I,2K,2L,2M,2N,2P,2Q,2R,2S,2T,2V,2W,2Y,2t,2y,3A,3C,3D,3E,3F,3G,3H,3I,3K,3L,3M,3N,3P,3Q,3R,3S,3T,3V,3W,3Y,3t,3y,4A,4C,4D,4E,4F,4G,4H,4I,4K,4L,4M,4N,4P,4Q,4R,4S,4T,4V,4W,4Y,4t,4y
4,1.252723,1.248830,0.873898,0.601646,1.047566,1.523919,1.680993,0.566336,1.786166,1.017873,1.297056,2.205684,1.478037,1.473652,2.121540,1.410493,1.146518,0.689077,0.971239,1.814270,0.580574,1.093674,1.357054,2.036879,0.416371,0.417932,0.612234,0.844795,1.129163,0.600633,3.882917,1.076347,0.670114,0.750983,0.528956,0.902214,8.277744,3.094029,2.293587,0.667872,0.600235,0.605753,0.662378,0.583033,1.033506,1.202165,0.421425,0.455981,0.568662,0.693936,1.079918,0.540987,3.777316,0.573559,0.575909,0.601408,0.672721,0.949034,19.928012,1.107675,1.014496,0.544039,0.580602,0.614292,0.684288,0.632425,1.488888,1.494032,0.740843,0.838902,0.907602,1.538154,1.33393,0.857336,2.570105,1.034472,1.151573,1.185649,1.260408,1.279404,3.04361,1.278347,1.277058,0.909421,0.874610,1.041559,0.975093,0.831319,1.317615,1.430100,0.62584,0.646126,1.122382,1.264056,1.248934,0.857198,2.007806,1.010056,1.050676,0.973416,1.234496,1.013656,3.025823,1.135926,1.092143,0.886697,1.093696,1.117178,0.935301,0.975119,0.840923,1.658432,0.518939,0.477760,1.896991,0.910117,1.009849,1.216862,1.086259,1.179772,1.437809,1.169001,0.769496,1.265725,1.198927,1.295663,1.273282,1.076143,1.035844,1.034681,0.823531,0.942467,1.437826,2.002882,0.720787,0.643082,0.752372,1.521882,1.965847,0.821316,1.488958,0.975453,0.894401,1.372951,1.034323,0.984906,1.578773,4.191876,1.996739,1.028776,0.808967,0.982034,1.230224,1.264340,1.296462,1.892505,0.716739,0.711202,1.047595,1.544028,1.693936,0.751384,2.174711,1.045462,0.895297,1.119572,1.246545,1.174084,2.219066,1.569695,1.308552,0.904983,1.081797,1.080408,0.984368,1.225548,1.237134,1.606234,0.823663,0.796645,0.911979,1.644231,1.546106,0.920043,2.630636,0.986489,1.090272,1.419044,2.203153,1.485343,2.254299,1.756481,1.775730,0.857565,0.944533,1.024330,1.24821,0.950655
5,1.294072,1.153273,0.993227,0.519128,1.161256,1.899052,1.748892,0.364165,2.444212,1.107083,1.654374,2.678062,1.689569,1.531418,2.411177,1.527736,1.266838,0.596234,0.960301,2.166737,0.413546,1.167212,1.506838,1.504885,0.246259,0.234297,0.527537,0.815138,1.923967,0.451609,4.974254,1.273318,0.644940,0.615123,0.466501,0.706624,12.146700,3.648219,2.983792,0.534407,0.417579,0.537104,0.484827,0.433791,1.074513,0.920832,0.262625,0.278554,0.338557,0.665245,1.131245,0.263748,3.034166,0.332554,0.361946,0.486353,0.820732,1.013150,23.083859,1.046747,0.807574,0.296889,0.441138,0.448077,0.515564,0.449338,1.515622,1.229415,0.667259,0.758091,0.829927,1.412425,1.34551,0.699594,2.970424,0.876682,1.131393,1.145324,1.443855,1.386143,4.18280,1.399942,1.155929,0.868439,0.867052,0.987613,0.831115,0.729234,1.586096,1.221776,0.58173,0.547084,0.841509,1.329727,1.479166,0.722431,2.699287,0.697416,0.812174,0.919091,1.500741,0.998190,3.716144,1.941354,1.325112,0.867442,1.061004,0.942139,0.683293,0.773865,0.787243,1.547058,0.433654,0.371891,2.560955,0.883633,1.096996,1.467718,1.097079,1.294964,1.807920,1.147164,0.464697,1.021469,1.243349,1.603971,1.381737,1.397411,1.127963,1.187843,0.651683,0.733107,1.609683,2.148273,0.795194,0.547055,0.846036,1.829739,2.922700,1.026199,1.853814,1.401466,1.098968,1.626895,1.058891,1.016294,2.588813,4.498148,2.240851,1.454272,1.302340,1.153029,0.656788,0.931439,1.244741,1.687986,0.711769,0.666914,1.182510,1.691920,2.086424,1.346346,2.725641,1.251046,1.030460,1.307980,1.208911,1.408288,2

In [ ]:
y_pred.head(2) # predicted values

,-1A,-1C,-1D,-1E,-1F,-1G,-1H,-1I,-1K,-1L,-1M,-1N,-1P,-1Q,-1R,-1S,-1T,-1V,-1W,-1Y,-1t,-1y,-2A,-2C,-2D,-2E,-2F,-2G,-2H,-2I,-2K,-2L,-2M,-2N,-2P,-2Q,-2R,-2S,-2T,-2V,-2W,-2Y,-2t,-2y,-3A,-3C,-3D,-3E,-3F,-3G,-3H,-3I,-3K,-3L,-3M,-3N,-3P,-3Q,-3R,-3S,-3T,-3V,-3W,-3Y,-3t,-3y,-4A,-4C,-4D,-4E,-4F,-4G,-4H,-4I,-4K,-4L,-4M,-4N,-4P,-4Q,-4R,-4S,-4T,-4V,-4W,-4Y,-4t,-4y,-5A,-5C,-5D,-5E,-5F,-5G,-5H,-5I,-5K,-5L,-5M,-5N,-5P,-5Q,-5R,-5S,-5T,-5V,-5W,-5Y,-5t,-5y,1A,1C,1D,1E,1F,1G,1H,1I,1K,1L,1M,1N,1P,1Q,1R,1S,1T,1V,1W,1Y,1t,1y,2A,2C,2D,2E,2F,2G,2H,2I,2K,2L,2M,2N,2P,2Q,2R,2S,2T,2V,2W,2Y,2t,2y,3A,3C,3D,3E,3F,3G,3H,3I,3K,3L,3M,3N,3P,3Q,3R,3S,3T,3V,3W,3Y,3t,3y,4A,4C,4D,4E,4F,4G,4H,4I,4K,4L,4M,4N,4P,4Q,4R,4S,4T,4V,4W,4Y,4t,4y
4,0.880791,0.622719,0.960550,0.700666,1.153082,1.564817,1.056182,0.856165,2.141996,1.751584,1.589074,1.355316,1.320749,1.206049,2.315694,1.194068,0.868294,0.926967,0.850907,1.126364,0.721614,1.673116,1.140022,0.522095,0.594378,1.666814,0.854763,0.661071,1.290889,0.658254,3.097906,1.223099,0.988903,0.856534,1.004860,1.477508,7.259325,1.932906,1.623866,0.657790,0.624887,0.641342,1.948765,5.038746,1.082604,0.659705,0.815372,0.980333,0.727818,1.084473,1.400426,0.678640,3.807357,0.815253,0.600410,0.680822,0.935720,1.061968,10.941114,1.491273,1.030589,0.727280,0.671362,0.661280,1.525312,2.452796,1.233271,0.813517,0.619934,0.992625,1.043517,1.19998,1.401803,1.019787,2.054327,1.109080,1.257301,0.929303,1.121664,1.155277,2.710951,1.388477,1.488443,1.172090,1.118719,1.063543,1.648121,1.598501,1.122952,0.797036,0.658023,0.663709,1.361683,0.862730,1.304848,1.229473,1.567571,1.561108,1.063051,0.857515,1.200660,1.005065,2.137386,1.623627,1.831581,1.313838,1.04585,1.246158,1.298929,1.715210,0.662742,0.340296,0.043465,0.576465,3.101009,0.906655,1.285566,1.634424,1.160999,1.248203,1.464651,1.062558,3.968082,0.985305,1.307601,1.682637,1.185116,1.709194,1.507619,1.366714,1.796392,2.048392,1.593327,1.102411,1.378234,0.357327,1.073161,1.492891,1.593010,1.244568,1.641541,1.238622,0.822334,1.188205,1.378617,1.181676,3.078151,3.686029,2.058980,1.334777,0.382799,0.609451,1.281502,0.541642,1.285201,0.893367,0.892318,0.337786,1.384147,1.674513,1.384488,1.017258,2.380152,1.188938,1.155136,1.226855,1.802301,1.279146,2.279682,2.776567,1.584899,1.132408,0.840852,1.149841,1.152554,1.255384,1.371209,0.916174,1.228150,0.930996,1.051920,1.599864,1.578821,0.795891,2.198844,0.963892,0.980861,1.498177,1.948126,1.704162,1.934293,2.542421,1.580968,1.088194,1.482994,0.998058,3.797018,0.912856
5,0.929538,0.858041,1.000899,0.581660,1.043249,1.839039,1.365918,0.516358,2.073915,1.331279,1.562119,1.919175,1.276413,1.375619,2.547575,1.392574,1.078016,0.694548,0.958035,1.597075,0.477856,1.152724,1.397301,1.213398,0.128213,0.475064,0.376965,0.784733,1.132425,0.578571,3.133225,1.418803,0.713360,0.804273,0.225411,0.904714,9.750872,2.875341,2.452484,0.699616,0.346472,0.221861,1.175775,1.158697,0.926652,0.906074,0.480399,0.860973,0.528511,0.510841,1.292207,0.415131,3.373493,0.613372,0.504264,0.531332,0.619454,0.815627,13.341736,1.605012,1.128888,0.420068,0.605516,0.696449,1.201428,1.959073,1.201881,0.943274,0.867961,1.073280,1.044476,1.18035,1.436667,1.032511,2.404923,1.193812,1.223354,1.071209,0.924396,1.152283,3.067859,1.826459,1.721899,1.133352,1.548273,1.055927,1.951309,1.364031,1.097885,1.067915,0.879728,0.911181,1.213839,1.103597,1.305292,1.129559,1.799078,1.286660,1.108686,0.910434,1.167149,0.978838,2.780817,2.203076,1.483477,0.974390,0.97109,1.190445,1.083757,1.079758,0.217424,1.118824,0.228350,0.404896,3.125341,1.976914,1.104548,1.997904,0.998664,1.271509,1.969985,0.949745,0.632168,1.209517,1.319231,1.426457,1.527416,1.807208,1.511742,1.181756,0.733854,0.737861,1.190224,1.484794,0.565462,0.458537,0.813866,1.346953,2.039813,1.356840,1.489837,1.368404,0.964276,1.092393,0.672661,0.949666,3.101026,3.495048,3.253386,1.550002,0.965943,1.062792,0.601654,1.853414,1.159375,1.496792,0.937857,0.695079,1.807525,1.598659,2.118884,1.331331,1.117564,1.532495,1.448284,1.568873,0.768246,1.363604

In [ ]:
#| export
def predict_ml(df, # Dataframe that contains features
               feat_col, # feature columns
               model_pth # models.joblib
              ):
    
    test = df[feat_col]
    
    model = load(model_pth)
    
    pred = model.predict(test)
    
    pred_df = pd.DataFrame(pred)
    
    return pred_df

In [ ]:
# pred = predict_ml(X_test, feat_col, 'model.joblib')

In [ ]:
# pred.head()

In [ ]:
#| export
def score_all(target, pred):
    
    "Calculate the overall correlation between two dataframes; need to have same index and columns"
    
    # Calculate RMSE
    mse = mean_squared_error(target, pred)
    # rmse = math.sqrt(mse)
    print(f'mse is {mse:.4f}')

    # Calculate the Spearman correlation coefficient
    spearman_corr, _ = spearmanr(target.values.ravel(), pred.values.ravel())
    print(f"Spearman correlation coefficient: {spearman_corr:.4f}")

    # Calculate the Pearson correlation coefficient
    pearson_corr, _ = pearsonr(target.values.ravel(), pred.values.ravel())
    print(f"Pearson correlation coefficient: {pearson_corr:.4f} ")
    
    # return mse,spearman_corr, pearson_corr

### Cross-Validation

In [ ]:
#| export
def train_cv(df, # dataframe of values
             feat_col, # feature columns
             target_col,  # target columns
             splits, # splits
             model, # sklearn model
             save_name = None, # model name to be saved, e.g., 'LR'
             params = {}
            ):
    OOF = []
    for fold, split in enumerate(splits):
        print(f'------ fold: {fold} --------')
        
        if save_name is not None: 
            target, pred = train_ml(df, feat_col, target_col, split, model, f'models/{save_name}_fold.joblib',params)
        else:
            target, pred = train_ml(df, feat_col, target_col, split, model, params=params)

        score_all(target,pred)
        OOF.append(pred)
        
    oof_df = pd.concat(OOF).sort_index()
    
    return oof_df

In [ ]:
oof = train_cv(df,feat_col, target_col, splits, LinearRegression())

------ fold: 0 --------
LinearRegression()
mse is 1.1492
Spearman correlation coefficient: 0.4699
Pearson correlation coefficient: 0.5758 
------ fold: 1 --------
LinearRegression()
mse is 1.4311
Spearman correlation coefficient: 0.4784
Pearson correlation coefficient: 0.5117 
------ fold: 2 --------
LinearRegression()
mse is 0.9352
Spearman correlation coefficient: 0.5135
Pearson correlation coefficient: 0.6516 
------ fold: 3 --------
LinearRegression()
mse is 1.0260
Spearman correlation coefficient: 0.4896
Pearson correlation coefficient: 0.6412 
------ fold: 4 --------
LinearRegression()
mse is 0.9092
Spearman correlation coefficient: 0.4865
Pearson correlation coefficient: 0.6101 


In [ ]:
oof.head(2)

,-1A,-1C,-1D,-1E,-1F,-1G,-1H,-1I,-1K,-1L,-1M,-1N,-1P,-1Q,-1R,-1S,-1T,-1V,-1W,-1Y,-1t,-1y,-2A,-2C,-2D,-2E,-2F,-2G,-2H,-2I,-2K,-2L,-2M,-2N,-2P,-2Q,-2R,-2S,-2T,-2V,-2W,-2Y,-2t,-2y,-3A,-3C,-3D,-3E,-3F,-3G,-3H,-3I,-3K,-3L,-3M,-3N,-3P,-3Q,-3R,-3S,-3T,-3V,-3W,-3Y,-3t,-3y,-4A,-4C,-4D,-4E,-4F,-4G,-4H,-4I,-4K,-4L,-4M,-4N,-4P,-4Q,-4R,-4S,-4T,-4V,-4W,-4Y,-4t,-4y,-5A,-5C,-5D,-5E,-5F,-5G,-5H,-5I,-5K,-5L,-5M,-5N,-5P,-5Q,-5R,-5S,-5T,-5V,-5W,-5Y,-5t,-5y,1A,1C,1D,1E,1F,1G,1H,1I,1K,1L,1M,1N,1P,1Q,1R,1S,1T,1V,1W,1Y,1t,1y,2A,2C,2D,2E,2F,2G,2H,2I,2K,2L,2M,2N,2P,2Q,2R,2S,2T,2V,2W,2Y,2t,2y,3A,3C,3D,3E,3F,3G,3H,3I,3K,3L,3M,3N,3P,3Q,3R,3S,3T,3V,3W,3Y,3t,3y,4A,4C,4D,4E,4F,4G,4H,4I,4K,4L,4M,4N,4P,4Q,4R,4S,4T,4V,4W,4Y,4t,4y
0,0.985162,0.888783,0.650083,0.470425,1.453643,1.411065,1.417298,0.920627,2.458418,1.262223,1.503112,1.263942,1.93209,1.094558,2.081244,0.887029,0.975983,0.804702,1.167784,1.839061,0.638824,0.700638,1.519925,1.726393,0.852782,0.801714,0.881199,1.153690,1.232933,0.799700,1.638345,1.084671,1.812468,1.129932,1.980593,2.331065,3.628460,2.179570,2.055677,0.804283,0.745818,0.849442,0.438380,0.356337,1.532111,1.332136,-0.003621,0.093122,1.056355,0.994271,0.765361,1.246958,1.485118,0.922334,1.500040,0.976653,2.062295,0.916558,2.301953,1.300145,1.380309,0.954043,0.830219,0.979465,-0.563371,0.223295,1.332589,1.256111,0.576807,0.750933,1.111451,1.440795,0.986112,1.213366,1.581206,1.266443,1.149016,0.770259,1.584879,1.085441,1.753005,1.132172,1.211919,1.258714,1.021737,1.229256,0.113481,0.321791,0.917841,1.537907,0.286272,0.358621,1.253529,1.125725,1.174129,2.255844,0.890841,1.484774,1.409381,0.707453,1.358767,0.934388,2.043394,0.870646,1.149838,1.348048,1.448632,2.203252,0.195100,0.809193,0.869675,1.023177,0.879266,1.140842,0.90017,12.898273,0.293536,0.637649,0.826959,0.889004,0.515031,0.717413,2.993179,0.504604,1.040342,1.100091,1.443135,0.655120,0.522426,0.383007,1.117580,0.152828,1.181576,1.318348,1.444544,1.271309,1.417365,1.583491,1.750619,0.762573,1.204545,1.001189,1.161987,1.892462,1.151114,1.348847,1.520310,0.783146,4.070170,0.697158,1.372698,1.836292,1.146508,0.218075,1.132494,1.749113,0.993369,1.179805,1.380494,1.840563,1.459534,0.878186,2.463665,1.031125,0.989562,1.503613,1.817318,1.366825,1.705683,0.983024,4.478827,1.090159,1.341009,1.405892,0.912008,0.514723,1.056797,1.635825,1.028232,1.506865,1.458827,1.367372,1.281341,0.972839,1.955609,0.860715,1.225984,1.580088,1.861708,1.434961,2.290511,1.074263,2.419911,0.785828,1.784527,1.556943,2.697593,0.759031
1,0.860345,1.056054,-0.021727,0.733150,1.060984,0.664783,1.325944,0.880354,1.344422,1.336841,1.505645,1.433817,1.58512,1.343445,0.987193,0.875764,1.298810,0.934836,0.738402,1.175065,0.547596,0.800354,0.191479,1.098150,2.503217,6.077910,0.292255,0.382276,0.740177,0.624708,1.862604,0.778881,0.866448,0.608232,1.039540,1.084842,2.223496,2.411831,1.508868,0.650832,-0.140831,-0.134454,1.640363,0.989736,0.835254,0.890961,1.304374,0.466219,0.662810,0.589999,0.879610,0.713971,1.750399,0.402877,0.596547,1.199719,1.222576,1.138276,8.787333,1.505660,1.283365,0.780118,0.627713,0.746536,2.194788,0.426225,1.028474,1.172266,1.228348,0.953209,0.832176,1.299877,0.848722,0.658576,1.356633,0.591675,0.885460,0.993212,1.686173,0.978212,1.916378,1.488129,1.282300,0.903033,1.146155,0.883491,0.569094,0.927771,0.816491,1.030750,0.779430,0.381243,0.993179,1.161713,1.900947,0.974397,1.467198,0.884084,0.885221,0.855353,1.224898,0.920743,2.338325,0.137174,0.955270,0.749451,1.626050,0.972339,1.071668,1.101694,0.610453,1.300101,0.447875,1.836936,0.78038,2.999725,0.988816,1.302223,0.400237,1.372531,0.995811,1.213511,3.300998,1.177625,0.566577,1.339680,1.652790,1.072424,0.664591,0.744962,1.973112,1.848135,1.252339,1.400457,1.808257,0.860343,0.749320,2.415468,1.691803,0.460353,0.818736,0.179111,0.397072,1.005807,1.805958,1.172563,0.290194,2.229342,-0.937808,0.780147,0.660023,1.123837,1.017262,2.576274,1.049215,1.089985,0.971367,0.330528,0.779600,2.011519,1.396939,0.713822,1.528657,0.395893,0.544543,1.122328,2.057665,1.1

In [ ]:
oof.shape

(303, 198)

### Score

In [ ]:
y = df[target_col]

score_all(y, oof)

mse is 1.0909
Spearman correlation coefficient: 0.4875
Pearson correlation coefficient: 0.5931 


In [ ]:
#| export
def score_each(target, 
               pred,
               absolute=False, # If absolute, then will get absolute value of spearman and pearson
              ):
    "Calculate spearman and pearson per row"
    
    # pred.columns = target.columns
    sp = target.corrwith(pred,axis=1,method='spearman')
    
    pear = target.corrwith(pred,axis=1,method='pearson')
    
    df = pd.DataFrame(np.stack([sp,pear]).T,columns = ['spearman','pearson'])
    if absolute ==True:
        df = df.apply(abs)
        
    print(f'average spearman for each row is {df.spearman.mean()}')
    print(f'average pearson for each row is {df.pearson.mean()}')
    return df

In [ ]:
cor = score_each(y,oof)

average spearman for each row is 0.5016147601026477
average pearson for each row is 0.5844748750730399


In [ ]:
cor.head()

,spearman,pearson
0,0.726783,0.912949
1,0.191249,0.335579
2,0.241742,0.358883
3,0.853006,0.964424
4,0.549874,0.824573


## XGB

In [ ]:
#| export
def xgb_trainer(df,
                feature_col,
                target_col,
                test_index=None,
                xgb_params = { 
                            'max_depth':7, #from 4 to 7
                            'learning_rate':0.001, #from 0.001
                            'subsample':0.8,
                            'colsample_bytree':1, # from 0.2 to 1, because need to take all features
                            'eval_metric':'rmse',
                            'objective':'reg:squarederror',
                            'tree_method':'gpu_hist',
                            'predictor':'gpu_predictor',
                            'random_state':123
                        },
                model_file='xgb_model.bin',
                split_seed = 123, # seed of random split
               ):
    
    X = df[feature_col]
    y = df[target_col]
    
    print(f'xgb params is: {xgb_params}')
    
    if test_index is None:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=split_seed)
    else:
        X_train,y_train = X.loc[~X.index.isin(test_index)],y.loc[~X.index.isin(test_index)]
        X_test, y_test = X.loc[test_index],y.loc[test_index]

        
    print(X_train.shape,y_train.shape,X_test.shape, y_test.shape)
    print(y_test.index)
    #prepare matrix for xgb
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test, y_test)
    
    model = xgb.train(xgb_params, 
            dtrain=dtrain,
            evals=[(dtrain,'train'),(dtest,'valid')],
            num_boost_round=9999,
            early_stopping_rounds=100,
            verbose_eval=100,)
    
    # Save the model
    path = Path(model_file)
    
    # Make a directory if not exists
    path.parent.mkdir(exist_ok=True)
        
    model.save_model(model_file)
    print(f'Model saved to {model_file}')
    
    # Prepare the pred/target df
    pred = model.predict(dtest)
    
    out = np.vstack([np.ravel(y_test),np.ravel(pred)]).T
    pred_df = pd.DataFrame(out,index=y_test.index, columns = ['target','pred'] )
    
    spearman_corr, _ = spearmanr(pred_df.target, pred_df.pred)
    print(f'Spearman correlation: {spearman_corr:.2f}')
    pearson_corr, p_value = pearsonr(pred_df.target, pred_df.pred)
    print(f'Pearson correlation: {pearson_corr:.2f}')


    
    fig, ax = plt.subplots()
    ax.scatter(pred_df.target, pred_df.pred)
    ax.set_xlabel('True values')
    ax.set_ylabel('Predicted values')
    ax.set_title('Scatter plot of true versus predicted values')
    plt.show()
    plt.close()
    
    
    dd = model.get_score(importance_type='gain')
    gain = pd.DataFrame({'feature':dd.keys(),f'gain_importance':dd.values()}).set_index('feature').sort_values(by='gain_importance',ascending=False)
    gain[:10].plot.barh()
    plt.show()
    plt.close()
        
    dd = model.get_score(importance_type='weight')
    weight = pd.DataFrame({'feature':dd.keys(),f'weight_importance':dd.values()}).set_index('feature').sort_values(by='weight_importance',ascending=False)
    weight[:10].plot.barh()
    plt.show()
    plt.close()
    
    return pred_df, gain, weight

In [ ]:
# xgb_trainer(df,feat_col, target_col)

In [ ]:
#| export
def xgb_predict(df, # a dataframe that contains ID and features for prediction
                feature_col, #feature column name
                ID_col = "ID", #ID column name
                model_file='xgb_model.bin'):
    # Load the XGBoost model
    model = xgb.Booster()
    model.load_model(model_file)
    
    # Prepare data for prediction
    X = df[feature_col]
    dtest = xgb.DMatrix(X)
    
    # Make predictions
    preds = model.predict(dtest)
    
    # Combine predictions with IDs into a DataFrame
    result_df = pd.DataFrame({ID_col: df[ID_col], 'preds': preds})
    
    return result_df

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()